In [1]:
#!pip install pandas
#!pip install xgboost
#!pip install -U scikit-learn
#!pip install cupy-cuda12x

In [6]:
import pandas as pd 
import numpy as np
import cupy as cp

from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from utils import *

# optimization
import optuna

from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)


df = pd.read_csv('../data/exchange_rate_imputed.csv')
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
columns = df.columns.tolist()

target = '6'
target_mean = df[target].mean()
target_std = df[target].std()

df.head(3)

,0,1,2,3,4,5,6,OT
date,,,,,,,,
1990-01-01,0.7855,1.6110,0.861698,0.634196,0.211242,0.006838,0.525486,0.595
1990-01-02,0.7818,1.6100,0.861104,0.633513,0.211242,0.006863,0.523972,0.595
1990-01-03,0.7867,1.6293,0.861030,0.648508,0.211242,0.006975,0.526316,0.595


In [ ]:
def print_trial_info(study, trial):
    print(f"Trial {trial.number}: Value = {trial.value}")

pruner = optuna.pruners.SuccessiveHalvingPruner()
study = optuna.create_study(direction='minimize', pruner=pruner)
study.optimize(objective, n_trials=2, callbacks=[print_trial_info])

In [ ]:
x_forecaste = create_lags(df_norm, lags=lag-1).iloc[-1]
x_forecaste = x_forecaste.to_numpy()
x_forecaste = np.expand_dims(x_forecaste, axis=0)

y_forecast = regressor.predict(x_forecaste)
y_forecast = [(x * np.sqrt(normlizer.var_[6])) + normlizer.mean_[6] for x in y_forecast]
y_forecast = np.array(y_forecast).flatten()

x_forecaste.shape,y_forecast.shape

## save prediction

In [26]:
submission = pd.DataFrame(y_forecast, columns=['Prediction'])
submission['Id'] = range(100)
submission = submission[['Id', 'Prediction']]
submission.to_csv('submission.csv', index=False)

In [16]:
!kaggle competitions submit -c time-series-classification-part-2new -f submission.csv -m "XGBoost with 2 lags untuned"

Successfully submitted to Time Series Classification: Part 2



  0%|          | 0.00/1.40k [00:00<?, ?B/s]
100%|██████████| 1.40k/1.40k [00:00<00:00, 2.72kB/s]
